In [1]:
import tensorflow as tf

import numpy as np
import os
import time

In [2]:
path_to_file = "/Users/AnaPSilva/Documents/Ana/Ironhack/Bootcamp/Final_Project/Data/Rap/50cent.txt"

In [3]:
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print(f'Length of text: {len(text)} characters')

Length of text: 214884 characters


In [4]:
# Take a look at the first 250 characters in text
print(text[:250])

21 QUESTIONS
[50 Cent]
  New York City!
  You are now rapping...with 50 Cent
  You gotta love it...
  I just wanna chill and twist a lot
  Catch suns in my 7 45
  You drive me crazy shorty I
  Need to see you and feel you next to me
  I prov


In [5]:
# The unique characters in the file
vocab = sorted(set(text))
print(f'{len(vocab)} unique characters')

81 unique characters


In [6]:
## Before training, you need to convert the strings to a numerical representation.
## convert each character into a numeric ID. 
## It just needs the text to be split into tokens first.
example_texts = ['abcdefg', 'xyz']

chars = tf.strings.unicode_split(example_texts, input_encoding='UTF-8')
chars

2022-03-13 02:13:58.240783: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

In [7]:
## from tokens to character IDs
ids_from_chars = tf.keras.layers.StringLookup(
    vocabulary=list(vocab), mask_token=None)
ids = ids_from_chars(chars)
ids

<tf.RaggedTensor [[56, 57, 58, 59, 60, 61, 62], [79, 80, 81]]>

In [8]:
## it will also be important to invert this representation and 
## recover human-readable strings from it.
chars_from_ids = tf.keras.layers.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)

In [9]:
chars = chars_from_ids(ids)
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

In [10]:
## join the characters back into strings.
tf.strings.reduce_join(chars, axis=-1).numpy()

array([b'abcdefg', b'xyz'], dtype=object)

In [11]:
def text_from_ids(ids):
  """join the characters back into strings"""
  return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

#### Create training examples and targets

In [12]:
## convert the text vector into a stream of character indices.
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
all_ids

<tf.Tensor: shape=(214884,), dtype=int64, numpy=array([17, 16,  4, ...,  3,  1,  2])>

In [13]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)
for ids in ids_dataset.take(10):
    print(chars_from_ids(ids).numpy().decode('utf-8'))

2
1
 
Q
U
E
S
T
I
O


In [14]:
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)
examples_per_epoch

2127

In [15]:
## The batch method lets you easily convert 
## these individual characters to sequences of the desired size.
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

for seq in sequences.take(1):
  print(chars_from_ids(seq))

tf.Tensor(
[b'2' b'1' b' ' b'Q' b'U' b'E' b'S' b'T' b'I' b'O' b'N' b'S' b'\r' b'\n'
 b'[' b'5' b'0' b' ' b'C' b'e' b'n' b't' b']' b'\r' b'\n' b' ' b' ' b'N'
 b'e' b'w' b' ' b'Y' b'o' b'r' b'k' b' ' b'C' b'i' b't' b'y' b'!' b'\r'
 b'\n' b' ' b' ' b'Y' b'o' b'u' b' ' b'a' b'r' b'e' b' ' b'n' b'o' b'w'
 b' ' b'r' b'a' b'p' b'p' b'i' b'n' b'g' b'.' b'.' b'.' b'w' b'i' b't'
 b'h' b' ' b'5' b'0' b' ' b'C' b'e' b'n' b't' b'\r' b'\n' b' ' b' ' b'Y'
 b'o' b'u' b' ' b'g' b'o' b't' b't' b'a' b' ' b'l' b'o' b'v' b'e' b' '
 b'i' b't' b'.'], shape=(101,), dtype=string)


In [16]:
## It's easier to see what this is doing if you join the 
## tokens back into strings

for seq in sequences.take(5):
  print(text_from_ids(seq).numpy())

b'21 QUESTIONS\r\n[50 Cent]\r\n  New York City!\r\n  You are now rapping...with 50 Cent\r\n  You gotta love it.'
b'..\r\n  I just wanna chill and twist a lot\r\n  Catch suns in my 7 45\r\n  You drive me crazy shorty I\r\n  N'
b'eed to see you and feel you next to me\r\n  I provide everything you need and I\r\n  Like your smile I do'
b"n''t wanna see you cry\r\n  Got some questions that I got to ask and I\r\n  Hope you can come up with the"
b" answers babe\r\n  [Nate Dogg]\r\n  Girl...It''s easy to   love me now\r\n  Would you love me if   I was do"


In [17]:
def split_input_target(sequence):
    """takes a sequence as input, duplicates, 
    and shifts it to align the input and label for each timestep"""
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

In [18]:
dataset = sequences.map(split_input_target)
for input_example, target_example in dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())

Input : b'21 QUESTIONS\r\n[50 Cent]\r\n  New York City!\r\n  You are now rapping...with 50 Cent\r\n  You gotta love it'
Target: b'1 QUESTIONS\r\n[50 Cent]\r\n  New York City!\r\n  You are now rapping...with 50 Cent\r\n  You gotta love it.'


#### Create training batches

In [19]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<PrefetchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

#### Build The Model

- **tf.keras.layers.Embedding:** The input layer. A trainable lookup table that will map each character-ID to a vector with embedding_dim dimensions;
- **tf.keras.layers.GRU:** A type of RNN with size units=rnn_units (You can also use an LSTM layer here.)
- **tf.keras.layers.Dense:** The output layer, with vocab_size outputs. It outputs one logit for each character in the vocabulary. These are the log-likelihood of each character according to the model.

In [20]:
# Length of the vocabulary in chars
vocab_size = len(vocab)
display(vocab_size)
# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

81

In [21]:
class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True)
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.gru.get_initial_state(x)
    x, states = self.gru(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x

In [22]:
model = MyModel(
    # Making sure the vocabulary size matches the `StringLookup` layers.
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

#### Try the model

In [23]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 82) # (batch_size, sequence_length, vocab_size)


In [24]:
model.summary()

Model: "my_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  20992     
                                                                 
 gru (GRU)                   multiple                  3938304   
                                                                 
 dense (Dense)               multiple                  84050     
                                                                 
Total params: 4,043,346
Trainable params: 4,043,346
Non-trainable params: 0
_________________________________________________________________


In [25]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()

## This gives us, at each timestep, a prediction of the next character index
sampled_indices

array([17, 49, 63, 25, 48, 26, 53,  4, 73, 65, 25, 27, 35, 43, 81, 16, 73,
       75,  1, 31, 42, 48, 48, 74, 37, 79, 16, 52, 20, 60, 19,  1, 71, 33,
       62, 64, 60, 64, 16, 65, 58, 46,  6,  8, 35, 16, 51, 60, 29, 18, 52,
       40, 74, 77, 12, 80, 44, 68, 32, 30, 52, 75, 47, 78,  7,  7, 59, 33,
       45, 76, 33, 16, 12, 41, 55, 39, 60, 48, 27, 57, 47, 72, 50, 43, 52,
       63, 21, 14, 54, 35, 48, 61,  6,  4, 68,  9,  7, 23, 35, 38])

In [26]:
## Decode these to see the text predicted by this untrained model
print("Input:\n", text_from_ids(input_example_batch[0]).numpy())
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy())

Input:
 b"hat we living in\r\n  Repeat 1 (2x)\r\n\r\n  [50 Cent]\r\n  Uh huh come on\r\n  Don''t nobody smile when I com"

Next Char Predictions:
 b"2Vh:U;Z rj:?HPz1rt\nDOUUsJx1Y5e4\npFgiei1jcS$'H1XeB3YMsv,yQmECYtTw&&dFRuF1,N]LeU?bTqWPYh6/[HUf$ m(&8HK"


#### Train the model
- At this point the problem can be treated as a standard classification problem. Given the previous RNN state, and the input this time step, predict the class of the next character.

In [27]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [28]:
example_batch_mean_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", example_batch_mean_loss)

Prediction shape:  (64, 100, 82)  # (batch_size, sequence_length, vocab_size)
Mean loss:         tf.Tensor(4.4065986, shape=(), dtype=float32)


In [29]:
tf.exp(example_batch_mean_loss).numpy()

81.990105

In [30]:
## training procedure using the tf.keras.Model.compile method. 
## Use tf.keras.optimizers.Adam with default arguments and the loss function.
model.compile(optimizer='adam', loss=loss)

In [31]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_rap_50cent_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [32]:
EPOCHS = 200
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/200
33/33 [==============================] - 128s 4s/step - loss: 3.8506
Epoch 2/200
33/33 [==============================] - 135s 4s/step - loss: 2.8871
Epoch 3/200
33/33 [==============================] - 121s 4s/step - loss: 2.4598
Epoch 4/200
33/33 [==============================] - 109s 3s/step - loss: 2.2783
Epoch 5/200
33/33 [==============================] - 108s 3s/step - loss: 2.1717
Epoch 6/200
33/33 [==============================] - 108s 3s/step - loss: 2.0750
Epoch 7/200
33/33 [==============================] - 109s 3s/step - loss: 1.9838
Epoch 8/200
33/33 [==============================] - 106s 3s/step - loss: 1.8993
Epoch 9/200
33/33 [==============================] - 107s 3s/step - loss: 1.8244
Epoch 10/200
33/33 [==============================] - 107s 3s/step - loss: 1.7529
Epoch 11/200
33/33 [==============================] - 120s 4s/step - loss: 1.6917
Epoch 12/200
33/33 [==============================] - 102s 3s/step - loss: 1.6372
Epoch 13/200
33/33 [=====

2022-03-13 04:24:40.929190: W tensorflow/core/framework/op_kernel.cc:1745] OP_REQUIRES failed at save_restore_v2_ops.cc:136 : RESOURCE_EXHAUSTED: training_checkpoints/ckpt_rap_50cent_65_temp/part-00000-of-00001.data-00000-of-00001.tempstate13932531981855091814; No space left on device


ResourceExhaustedError: training_checkpoints/ckpt_rap_50cent_65_temp/part-00000-of-00001.data-00000-of-00001.tempstate13932531981855091814; No space left on device [Op:SaveV2]

In [33]:
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, states

In [34]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

In [35]:
start = time.time()
states = None
next_char = tf.constant(['a'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

ah, uh ha, aye Dre
  You got me feelin real bulletproof up in this motherfucker
  Cuz my windows on some hoes (heah)
  You retire me wireds are ten in the hood
    Cause a nigga from change, Wance really ran''d fame so was put it this is
  I ain''t gotta steal from the corna'' store no mo''s no more
   Howa lottly game was a cure
  I''m hard for cais to put in workD
  and for his real you on some movie shit
(50 Cent)
My adoued my baby bring they wanna see you criss
  Sobe whute are rem gross the duck
  Ithal the first Lluh, I aint the niggaz of bombar
  Nigga ain''t nuttin change
  You move I''ll blow your brains
  Yea
  Lloyd Banks nigga
  Whats up nigga?
  Anytime niggas ford the same
  thing you shit to you how to run rap cuz Dr. Dre got the recipe (the recipe, recipe)
  Yea...Ailift nigga
  You fuckin wit him right?
  I know you fuckin wit him cause I said you fucking bricks say
  hus ya undes resorned to stop frontin''
  You go to the dealership
  But you n 

_____________________

In [36]:
tf.saved_model.save(one_step_model, '50cent')

2022-03-13 10:09:59.886168: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: 50cent/assets


INFO:tensorflow:Assets written to: 50cent/assets
